In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
I_GPU = 3

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = str(I_GPU)

In [ ]:
# %load_ext autoreload
# %autoreload 2
import os
import sys
import numpy as np
import torch
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
# from torch_points3d.datasets.segmentation.scannet import ScannetDataset, IGNORE_LABEL
from torch_points3d.datasets.segmentation.multimodal.scannet import ScannetDatasetMM, IGNORE_LABEL

_ = torch.cuda.is_available()
_ = torch.cuda.memory_allocated()
torch.cuda.set_device(I_GPU)

# Load config

In [ ]:
from omegaconf import OmegaConf
from torch_points3d.utils.config import hydra_read

# Set root to the DATA drive, where the data was downloaded
# DATA_ROOT = '/mnt/fa444ffd-fdb4-4701-88e7-f00297a8e29b/projects/datasets/s3dis'  # ???
# DATA_ROOT = '/media/drobert-admin/DATA/datasets/s3dis'  # IGN DATA
# DATA_ROOT = '/media/drobert-admin/DATA2/datasets/s3dis'  # IGN DATA2
# DATA_ROOT = '/var/data/drobert/datasets/s3dis'  # AI4GEO
# DATA_ROOT = '/home/qt/robertda/scratch/datasets/s3dis'  # CNES
DATA_ROOT = '/raid/dataset/pointcloud/data/scannet'  # ENGIE

# XYZ+RGB
overrides = [
    'model_name=Res16UNet34-L4-early',
#    f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-20/18-43-13'}",  # 5cm
    f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-10-24/23-12-40'}",  # 3cm
    
    'voting_runs=1',
    'tracker_options.full_res=True',
 #   'tracker_options.make_submission=True',
    'precompute_multi_scale=False',
    'num_workers=4',
    'batch_size=3',
    f'cuda={I_GPU}',
    'weight_name=miou',
#     'weight_name=last',
]

# # XYZRGB
# overrides = [
#     'model_name=Res16UNet34',
#     f"checkpoint_dir={'/workspace/projects/torch-points3d/outputs/2021-11-05/23-21-24'}",  # 3cm
#     'voting_runs=1',
#     'tracker_options.full_res=True',
#  #   'tracker_options.make_submission=True',
#     'precompute_multi_scale=False',
#     'num_workers=4',
#     'batch_size=4',
#     f'cuda={I_GPU}',
# #     'weight_name=miou',
#     'weight_name=last',
# ]

cfg = hydra_read(overrides, config_name='eval')

OmegaConf.set_struct(cfg, False)

# print(OmegaConf.to_yaml(cfg))# Load S3DIS dataset

In [ ]:
from torch_points3d.trainer import Trainer

trainer = Trainer(cfg)

test_transform = trainer._dataset.test_dataset[0].transform
test_transform_image = trainer._dataset.test_dataset[0].transform_image
val_transform = trainer._dataset.val_dataset.transform
val_transform_image = trainer._dataset.val_dataset.transform_image
train_transform = trainer._dataset.train_dataset.transform
train_transform_image = trainer._dataset.train_dataset.transform_image

In [ ]:
# Update the val and test transforms to match train transforms for inference-time augmentation
trainer._dataset.test_dataset[0].transform = test_transform
trainer._dataset.test_dataset[0].transform_image = test_transform_image

trainer._dataset.val_dataset.transform = val_transform
trainer._dataset.val_dataset.transform_image = val_transform_image

In [ ]:
# Update the val and test transforms to match train transforms for inference-time augmentation
trainer._dataset.test_dataset[0].transform = trainer._dataset.train_dataset.transform
trainer._dataset.test_dataset[0].transform_image = trainer._dataset.train_dataset.transform_image

trainer._dataset.val_dataset.transform = trainer._dataset.train_dataset.transform
trainer._dataset.val_dataset.transform_image = trainer._dataset.train_dataset.transform_image

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = 2097152 * 2

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = 1920000
# trainer._dataset.val_dataset.transform_image.transforms[2].credit = 1920000

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = 1920000 * 4
trainer._dataset.val_dataset.transform_image.transforms[2].credit = 1920000 * 4

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = 1920000 * 3
trainer._dataset.val_dataset.transform_image.transforms[2].credit = 1920000 * 3

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = 1920000 * 2
trainer._dataset.val_dataset.transform_image.transforms[2].credit = 1920000 * 2

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = 1920000
trainer._dataset.val_dataset.transform_image.transforms[2].credit = 1920000

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 / 4)
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 / 4)

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 / 2)
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 / 2)

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].use_coverage = False
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.test_dataset[0].transform_image.transforms[3].sigma = 0.02
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.saturation = [0.8, 1.2]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.brightness = [0.8, 1.2]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.contrast = [0.8, 1.2]

trainer._dataset.val_dataset.transform_image.transforms[2].use_coverage = False
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.val_dataset.transform_image.transforms[3].sigma = 0.02
trainer._dataset.val_dataset.transform_image.transforms[4].transform.saturation = [0.8, 1.2]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.brightness = [0.8, 1.2]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.contrast = [0.8, 1.2]

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].use_coverage = False
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.test_dataset[0].transform_image.transforms[3].sigma = 0.03
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.saturation = [0.6, 1.4]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.brightness = [0.6, 1.4]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.contrast = [0.6, 1.4]

trainer._dataset.val_dataset.transform_image.transforms[2].use_coverage = False
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.val_dataset.transform_image.transforms[3].sigma = 0.03
trainer._dataset.val_dataset.transform_image.transforms[4].transform.saturation = [0.6, 1.4]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.brightness = [0.6, 1.4]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.contrast = [0.6, 1.4]

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].use_coverage = False
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.test_dataset[0].transform_image.transforms[3].sigma = 0.01
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.saturation = [0.9, 1.1]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.brightness = [0.9, 1.1]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.contrast = [0.9, 1.1]

trainer._dataset.val_dataset.transform_image.transforms[2].use_coverage = False
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.val_dataset.transform_image.transforms[3].sigma = 0.01
trainer._dataset.val_dataset.transform_image.transforms[4].transform.saturation = [0.9, 1.1]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.brightness = [0.9, 1.1]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.contrast = [0.9, 1.1]

In [ ]:
trainer._dataset.test_dataset[0].transform_image.transforms[2].use_coverage = False
trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.test_dataset[0].transform_image.transforms[3].sigma = 0.005
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.saturation = [0.95, 1.05]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.brightness = [0.95, 1.05]
trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.contrast = [0.95, 1.05]

trainer._dataset.val_dataset.transform_image.transforms[2].use_coverage = False
trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 * 2)
trainer._dataset.val_dataset.transform_image.transforms[3].sigma = 0.005
trainer._dataset.val_dataset.transform_image.transforms[4].transform.saturation = [0.95, 1.05]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.brightness = [0.95, 1.05]
trainer._dataset.val_dataset.transform_image.transforms[4].transform.contrast = [0.95, 1.05]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.8, 1.2]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.8, 1.2]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.8, 1.2]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 * 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.9, 1.1]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.9, 1.1]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.9, 1.1]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.01
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.95, 1.05]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.95, 1.05]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.95, 1.05]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 4)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.001
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.99, 1.01]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.99, 1.01]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.99, 1.01]

In [ ]:
# trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
# trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 / 2)
# trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.2
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.1, 1.9]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.1, 1.9]
# trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.1, 1.9]

In [ ]:
# Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# trainer._dataset.test_dataset[0].transform_image.transforms[-1].mean = [0, 0, 0]
# trainer._dataset.test_dataset[0].transform_image.transforms[-1].std = [1, 1, 1]

In [ ]:
# trainer._cfg.tracker_options.full_res = False

In [ ]:
trainer._cfg.voting_runs = 1
# trainer._cfg.voting_runs = 3

In [ ]:
# Validation inference
trainer.eval(stage_name="val")

In [ ]:
# Test inference
#trainer.eval(stage_name="test")

# XYRGB*

In [ ]:
print(trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou'])

# Save full-resolution confusion matrix
# torch.save(trainer._tracker._full_confusion_matrix, '/raid/dataset/pointcloud/data/scannet/results/sparseconv3d_3cm.pt')

In [ ]:
# vote=1 - credit x3
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=1 - credit x3
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

# VOTE=1 - CREDIT x3
### 3 cm > 70.8 or more
### 5 cm > 70.3

In [ ]:
# vote=1 - credit x3
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=1 - credit x4
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=1 - credit x3
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=1 - credit x2
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=1
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=5
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3 - x2 credit
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3 - ÷2 credit
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

### MORALITY : NO AUGMENTATION, JUST PLAY WITH CREDIT AND NUMBER OF VOTES 

In [ ]:
# vote=3
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3 - ÷4 credit
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3 - x2 credit
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3 - x2 credit - softer train aug
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3 - ÷4 credit - softer train aug
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=3 - ÷2 credit - train aug
trainer._tracker.get_metrics(True)['val_miou'], trainer._tracker.get_metrics(True)['val_full_miou']

In [ ]:
# vote=1
trainer._tracker.get_metrics(True)

___

In [ ]:
# vote=3 - ÷4 credit - train aug
trainer._tracker.get_metrics(True)

In [ ]:
# vote=3 - ÷4 credit
trainer._tracker.get_metrics(True)